# Lecture 1b. Data Exploration Primer

Here we will explore the basics of Pandas with an example dataset from the Chicago Data Portal at https://data.cityofchicago.org. You should take some time to explore the portal and find your own datasets of interest to explore!

Recall from the first notebook that you can use the Pandas load_csv function. We've included that here but commented out that line and have loaded a local file instead to allow 

In [55]:
import pandas as pd

# Use this line as an example to load data directly from the City of Chicago Portal.
df = pd.read_csv('https://data.cityofchicago.org/api/views/5neh-572f/rows.csv?accessType=DOWNLOAD')

# We have saved the file locally so that you can load the file locally.
# df = pd.read_csv('/cta-ridership.csv')

In [56]:
df.head(5)

,station_id,stationname,date,daytype,rides
0,41280,Jefferson Park,12/22/2017,W,6104
1,41000,Cermak-Chinatown,12/18/2017,W,3636
2,40280,Central-Lake,12/02/2017,A,1270
3,40140,Dempster-Skokie,12/19/2017,W,1759
4,40690,Dempster,12/03/2017,U,499


Immediately from looking at this, we can get an understanding of the type of data we're looking at.  There's a station identifier and name, a date on which the statistic takes place, the day type, and the number of rides for that date.

It's not immediately clear what A/W, is, but looking at the description of the dataset here:
https://data.cityofchicago.org/Transportation/CTA-Ridership-L-Station-Entries-Daily-Totals/5neh-572f

tells us that this column indicates that we are looking at a weekday, weekend, or holiday.

We can then explore some basic characteristics of the data, including the size of the dataset, min/max/etc. to explore outliers, etc. This basic exploration allows us to spot potential outliers and mistakes in the data.

### What are some basic statistics about the data?

In [57]:
df.shape

(1232735, 5)

In [58]:
df.describe()

,station_id,rides
count,1.232735e+06,1.232735e+06
mean,4.076793e+04,2.979097e+03
std,4.499409e+02,3.061730e+03
min,4.001000e+04,0.000000e+00
25%,4.037000e+04,9.330000e+02
50%,4.076000e+04,1.940000e+03
75%,4.116000e+04,3.918000e+03
max,4.171000e+04,3.632300e+04


Some basic statistics: There is a station with no rides (minimum is zero!). Also, the station with the maximum number of rides appears to be about 10x the mean and median. Let's have a look at what those stations are.

In [25]:
df['stationname'].value_counts()

stationname
Racine                    8664
Montrose-Brown            8662
Western-Forest Park       8661
Monroe/Dearborn           8661
Montrose-O'Hare           8660
                          ... 
Cermak-McCormick Place    3531
Washington/State          2953
Washington/Wabash         2588
Damen-Lake                  62
Homan                       31
Name: count, Length: 149, dtype: int64

### Selection Based on Conditionals

Let's see which stations have the most and fewest rides, overall, and for particular types of days.

#### Station with the most rides

In [29]:
df[df['rides'] == max(df['rides'])].head(1)

,station_id,stationname,date,daytype,rides
912644,41320,Belmont-North Main,06/28/2015,U,36323


Interesting. The station that had the most number of rides was Belmont-North, on June 28, 2015. What might have caused that? (Hint: Do a quick Web search for June 28, 2015 to find out what happened on that date in Chicago.)

#### Station with the Least Rides

In [31]:
df[df['rides'] == 0].shape

(14805, 5)

Oops! There are 14805 station-date combinations with zero rides! Let's have a quick look to understand this further.

In [89]:
zero = df[df['rides'] == 0]
zero.head(5)

,station_id,stationname,date,daytype,rides
23,40200,Randolph/Wabash,12/25/2017,U,0
110,40640,Madison/Wabash,12/05/2017,W,0
333,40640,Madison/Wabash,12/03/2017,U,0
354,40640,Madison/Wabash,12/11/2017,W,0
423,40640,Madison/Wabash,12/30/2017,A,0


Looks like a lot of weekends and holidays. We can group by columns and types to get a better understanding of what might be going on. We can count how many dates a station had zero rides in the dataset and sort these in descending order.

In [35]:
zerogroups = zero.groupby(['stationname','daytype']).count()
zerogroups.sort_values(by=['date'],ascending=False)

,,station_id,date,rides
stationname,daytype,,,
Lawrence,W,866,866,866
Berwyn,W,866,866,866
Madison/Wabash,W,711,711,711
Washington/State,W,516,516,516
Skokie,U,390,390,390
...,...,...,...,...
Chicago/Franklin,W,1,1,1
Ashland-Lake,W,1,1,1
Skokie,W,1,1,1


**Note:** It should be clear from a little bit of research why some of the stations at the top of the list report zero dates. Do a little homework on some of them to find out!

### Exploring Temporal Patterns

First, let's figure out the date range that we're dealing with.

In [39]:
min(df['date'])

'01/01/2001'

In [40]:
max(df['date'])

'12/31/2023'

#### Create a Time Index

So we have all rides from January 1, 2001 to December 31, 2019. Let's do some statistics that group ride statistics by date. First we need to tell Pandas that the date column is in fact a date. So, we convert the column to a proper 'DateTime' type, and then set the index to this column.

This step takes a little bit of time!

In [42]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

Let's see what this does to our data. Now we can see that the date column is indexed, but the rows are not sorted.

In [44]:
df.head(10)

,station_id,stationname,daytype,rides
date,,,,
2017-12-22,41280,Jefferson Park,W,6104
2017-12-18,41000,Cermak-Chinatown,W,3636
2017-12-02,40280,Central-Lake,A,1270
2017-12-19,40140,Dempster-Skokie,W,1759
2017-12-03,40690,Dempster,U,499
2017-12-30,41660,Lake/State,A,8615
2017-12-17,40180,Oak Park-Forest Park,U,442
2017-12-02,40250,Kedzie-Homan-Forest Park,A,1353
2017-12-07,40120,35th/Archer,W,3353


#### Sort the Dataframe by Date

In [46]:
# Sort the columns by date
rides_by_date = df.sort_values(by='date')
rides_by_date.head(10)

,station_id,stationname,daytype,rides
date,,,,
2001-01-01,41120,35-Bronzeville-IIT,U,448
2001-01-01,40740,Western-Cermak,U,0
2001-01-01,40370,Washington/Dearborn,U,1039
2001-01-01,40330,Grand/State,U,2542
2001-01-01,40350,UIC-Halsted,U,273
2001-01-01,40220,Western-Forest Park,U,318
2001-01-01,41330,Montrose-O'Hare,U,383
2001-01-01,40650,North/Clybourn,U,1156
2001-01-01,40010,Austin-Forest Park,U,290


#### Sanity Checking

Looks good!  Now let's have a quick look at data for specific stations: the Garfield station at the Red Line and the Green lines, respectively.

In [48]:
garfield_red = rides_by_date[rides_by_date['stationname']=='Garfield-Dan Ryan']
garfield_red.head(14)

,station_id,stationname,daytype,rides
date,,,,
2001-01-01,41170,Garfield-Dan Ryan,U,1457
2001-01-02,41170,Garfield-Dan Ryan,W,3748
2001-01-03,41170,Garfield-Dan Ryan,W,3977
2001-01-04,41170,Garfield-Dan Ryan,W,4185
2001-01-05,41170,Garfield-Dan Ryan,W,4533
2001-01-06,41170,Garfield-Dan Ryan,A,3341
2001-01-07,41170,Garfield-Dan Ryan,U,1972
2001-01-08,41170,Garfield-Dan Ryan,W,4590
2001-01-09,41170,Garfield-Dan Ryan,W,4511


In [49]:
garfield_green = rides_by_date[rides_by_date['stationname']=='Garfield-South Elevated']
garfield_green.head(14)

,station_id,stationname,daytype,rides
date,,,,
2001-01-01,40510,Garfield-South Elevated,U,248
2001-01-02,40510,Garfield-South Elevated,W,617
2001-01-03,40510,Garfield-South Elevated,W,657
2001-01-04,40510,Garfield-South Elevated,W,697
2001-01-05,40510,Garfield-South Elevated,W,770
2001-01-06,40510,Garfield-South Elevated,A,443
2001-01-07,40510,Garfield-South Elevated,U,295
2001-01-08,40510,Garfield-South Elevated,W,695
2001-01-09,40510,Garfield-South Elevated,W,696


## Visualizing Timeseries Data

In [83]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


sns.set(rc={'figure.figsize':(11, 4)})
garfield_green['rides'].plot(linewidth=0.5)

<Axes: xlabel='date'>

Interesting!  You can see a jump in ridership at the Garfield Green Line ridership right at the same time there's a dip in the Garfield Red Line ridership. What happened?  Here's a clue: https://www.transitchicago.com/redsouth/

Let's now go back and see what happened to the Madison/Wabash station. And why there were so many zero values in the data.

In [78]:
rides_by_date[rides_by_date['stationname']=='Madison/Wabash']['rides'].plot(linewidth=0.5)

<Axes: xlabel='date'>

How do we know whether this is just a glitch in the dataset, or a real event? A little web searching can tell you a bit about this station. https://en.wikipedia.org/wiki/Madison/Wabash_station

"Madison/Wabash closed on March 16, 2015, after Sunday service in the Loop ceased for the night. The entrances were boarded up by morning-time, and trains started bypassing the station when Monday morning service started."